In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

from pprint import pprint

In [ ]:
from langchain_openai import ChatOpenAI
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"

llm = ChatOpenAI(model="gpt-4.1-mini")

Chroma DB를 사용할 때 발생할 수 있는 캐시 문제를 해결하기 위한 코드입니다.
```
{'messages': [ToolMessage(content="Error: ValueError('Could not connect to tenant default_tenant. Are you sure it exists?')\n Please fix your mistakes.", name='build_and_retrieve', tool_call_id='random_ID')]}
```
위 문제가 발생할 경우 아래 코드를 실행해주세요

In [ ]:
# 만약 이전에 크로마DB를 사용했다면 cache로 인해 오류가 날 수 있습니다
# 코드 진행 중 chromaDB로 인해 오류가 난다면 주석을 해제하고 해당 cell을 실행하여 cache를 삭제해 주세요

import chromadb.api

chromadb.api.client.SharedSystemClient.clear_system_cache()

# ToolNode 활용
이번 실습의 목표는 아래와 같습니다.

### 1. Tool node의 기본적인 활용 방법

- Tool node의 활용 방법을 익힙니다.
- Input을 직접 만들어도 되고, LLM을 활용하여 tool selection과 함께 Input을 구성할 수 있습니다.

### 2. Retrieval Tool 구축하기

- RAG를 tool의 형태로 변환하여 실행합니다.
- 상황에 따라 가변적으로 RAG를 할 때 용이합니다.

---

## 1. Tool node의 기본적인 활용 방법

LangGraph는 LLM Agent를 구현할 때 필요한 다양한 도구 및 구성요소를 제공합니다. 그 중에서 `ToolNode`는 LLM이 수행하기 힘든 업무를 대신해준다는 점에서 특히 중요한 역할을 합니다.
ToolNode의 특징은 다음과 같습니다.
- Agent가 사용할 수 있는 외부 'tool'을 정의하는 노드
    - input을 통해 정해진 tool을 실행, output을 제공
- LangGraph 그래프 내에서 이 도구들을 호출하고 결과를 처리할 수 있음
    - RunnableCallable 컴포넌트

### 기본 Tool 정의하기
Tool을 만들기 위해서는 `@tool` 데코레이터를 사용합니다. 데코레이터 패턴을 활용하면 함수를 tool로 간편하게 변환할 수 있습니다. 여기서는 두 가지 간단한 도구를 만들어 보겠습니다.

In [ ]:
# tool이라는 magic function을 활용하여 사용할 툴을 인지


@tool
def get_weather(location: str):
    """도시 이름을 입력받아 현재 날씨를 반환합니다."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 20 degrees and foggy."
    else:
        return "It's 30 degrees and sunny."


@tool
def get_coolest_cities():
    """멋진 도시 목록을 반환"""
    return "nyc, sf"


# 여러 개의 tool들을 리스트 형태로 제공
tools = [get_weather, get_coolest_cities]
# 다수의 tool을 갖고 있는 node가 생성되며, 인풋에 따라 사용할 tool 결정
tool_node = ToolNode(tools)

### LLM을 활용한 Tool 선택
Tool 실행 포맷을 생성하기 위해 LLM을 활용할 수 있습니다. LLM에게 도구 목록을 바인딩하면 자연어 입력을 적절한 도구 호출 포맷으로 변환해 줍니다.

In [ ]:
llm_with_tools = llm.bind_tools(tools)

LLM을 활용하면 사용자의 자연어 입력을 도구 호출 포맷으로 변환할 수 있어 편리합니다. 사용자가 "뉴욕의 날씨가 어때?"와 같이 물어보면, LLM이 이를 `get_weather("New York")`과 같은 도구 호출로 변환해 줍니다.

In [ ]:
output = llm_with_tools.invoke("what's the weather in sg?")
pprint(output)

다음과 같은 형태로 LLM이 output을 제공합니다. 이 형식은 도구 이름, 인자, ID 등을 포함하여 도구를 호출하는 데 필요한 정보를 담고 있습니다. 해당 포맷을 곧바로 tool_node에 제공하여 결과 값을 받을 수 있습니다.

```python
[{'name': 'get_weather', # tool 이름
  'args': {'location': 'San Francisco'}, # tool 함수 입력 인자
  'id': 'call_7kTlvOrWyBZtfmakyzTYQJU5', # tool 호출용 ID
  'type': 'tool_call'}] # 활성화 타입

```

In [ ]:
tool_node.invoke({"messages": [llm_with_tools.invoke("what's the weather in sf?")]})

In [ ]:
tool_node.invoke({"messages": [llm_with_tools.invoke("what's the weather in sg?")]})

In [ ]:
tool_node.invoke({"messages": [llm_with_tools.invoke("give me the coolest city in the world")]})

LLM을 사용하지 않고 직접 도구 호출 포맷을 만들 수도 있습니다. 특별한 전처리가 필요하거나 LLM 호출 없이 도구를 직접 실행하려는 경우에 유용합니다. 만약 LLM을 활용하지 않고 다른 preprocessing step을 추가하여 tool을 직접 실행하고자 한다면, 다음과 같이 `AIMessage`등 Message에 `tool_calls`인자를 다음과 같이 제공하는 방식으로 실행할 수도 있습니다.

In [ ]:
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[  # 인자에 Tool에 대한 정보 제공
        {
            "name": "get_weather",
            "args": {"location": "sf"},
            "id": "random_ID",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})

## 2. Retrieval Tool과 요약 및 답변 툴 구축하기

이제 Retrieval-Augmented Generation(RAG) 기능을 도구로 구현해보겠습니다. 다만 Langchain에서 만들었던 것과 달리 벡터DB에 데이터를 검색하여 회수하는 Tool과 정보를 요약하여 답변하는 Tool 두 개를 만듭니다.. 먼저 `build_and_retrieve` 도구를 만들고, 추가로 검색된 문서를 요약하는 도구`summarize_and_answer`도 구현해 보겠습니다.
구성된 도구를 `ToolNode`에 전달하여 검색 및 요약 작업을 수행할 수 있습니다.

In [ ]:
@tool
def build_and_retrieve(query: str):
    """문서 데이터베이스에서 근육 발달에 관련된 정보를 검색합니다"""
    loader = PyPDFLoader("Maximizing Muscle Hypertrophy.pdf")
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(pages)

    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    # 검색 결과를 가져옵니다
    docs = retriever.invoke(query)

    # 검색된 문서의 내용을 추출하여 가독성 있게 반환합니다
    results = []
    for i, doc in enumerate(docs):
        page_num = doc.metadata.get("page", "unknown")
        results.append(f"=== 검색결과 {i+1} (페이지 {page_num}) ===\n{doc.page_content}\n")

    # 결과를 하나의 문자열로 결합합니다
    return "\n".join(results)


@tool
def summarize_and_answer(query_and_documents: str):
    """검색된 문서 내용을 분석하여 질문에 대한 정확한 답변을 제공합니다"""

    # 입력값에서 질문과 문서를 분리
    parts = query_and_documents.split("DOCUMENTS:", 1)
    if len(parts) != 2:
        return "입력 형식이 올바르지 않습니다. '질문 DOCUMENTS: 문서내용' 형식으로 입력해주세요."

    query = parts[0].strip()
    documents = parts[1].strip()

    # LLM에게 문서 내용을 기반으로 질문에 답변하도록 요청
    prompt = f"""
    당신은 전문적인 건강 및 피트니스 조언자입니다. 다음 문서 내용을 바탕으로 질문에 답변해주세요.
    질문이나 문서에 언급되지 않은 내용은 추가하지 마세요.

    ## 질문
    {query}

    ## 문서 내용
    {documents}

    ## 지침
    1. 문서 내용만을 기반으로 답변하세요.
    2. 학술적인 참조가 있다면 포함해주세요.
    3. 답변은 3-5개의 핵심 포인트로 구성하고, 각 포인트는 1-2문장으로 간결하게 설명하세요.
    4. 마지막에 실용적인 조언으로 마무리하세요.
    5. 불확실한 정보가 있으면 언급하지 마세요.
    """

    # LLM을 사용하여 답변 생성
    response = llm.invoke(prompt)

    return response.content


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 20 degrees and foggy."
    else:
        return "It's 30 degrees and sunny."


@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "nyc, sf"


tools = [build_and_retrieve, summarize_and_answer, get_weather, get_coolest_cities]
tool_node = ToolNode(tools)

직접 input을 만들어서 tool_node를 실행할 수도 있습니다.

In [ ]:
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "build_and_retrieve",
            "args": {
                "query": "근육을 키우려면 식사 영양성분을 어떻게 구성해야해? 주의해야 할 음식이 있을까?? 내 데이터베이스를 검색하여 정보를 찾아줘"
            },
            "id": "random_ID",
            "type": "tool_call",
        }
    ],
)

output = tool_node.invoke({"messages": [message_with_single_tool_call]})
output

검색 결과의 내용을 확인해보면, 일부 문장에서 지방(fats), 심혈관계 질환(cardiovascular disease), 단백질(protein) 등의 단어가 보입니다.

In [ ]:
pprint(output["messages"][0].content)

In [ ]:
chromadb.api.client.SharedSystemClient.clear_system_cache()

### LLM과 도구 연동하기
이제 LLM에 도구를 바인딩하여 자연어 입력을 도구 호출로 변환해 보겠습니다.  
  
다만, 경우에 따라 LLM이 도구 사용법을 제대로 파악하지 못하거나(LLM의 문맥 이해 능력 미숙 및 도구에 대한 불충분한 설명) 비슷한 내용의 도구가 여러 개 있다면, 우리가 의도한 기능을 잘 활용하지 못할 수 있습니다.  
  
이 경우 작동이 잘 되지 않을 경우 프롬프트를 수정하여 해당 도구를 잘 사용할 수 있게 해야합니다.

In [ ]:
llm_with_tools = llm.bind_tools(tools)

도구와 관련 없는 질문이 들어올 경우, LLM은 일반적인 대화 모델처럼 응답합니다.

In [ ]:
# tool과 상관 없는 input이 들어올 경우, 일반 chat 모델과 동일한 결과 생성
output = llm_with_tools.invoke("What is the best way to learn art?")
output

이 경우 도구 호출이 발생하지 않았으므로 tool_calls 속성은 비어 있습니다.


In [ ]:
# tool과 상관 없는 Input이기 때문에 tool calling을 실행하지 않음
# tool calling은 정의한 tool의 description을 바탕으로 LLM이 선택
output.tool_calls

도구와 관련된 질문이 들어오면 LLM은 적절한 도구를 선택하여 호출합니다.

In [ ]:
tool_input = llm_with_tools.invoke("근육을 빠르게 키우는 방법이 뭐야? 내 데이터베이스를 기반으로 답변해")
tool_input.tool_calls

선택된 도구를 실행합니다.

In [ ]:
tool_output = tool_node.invoke({"messages": [tool_input]})
tool_output

결과를 확인합니다.

In [ ]:
pprint(tool_output["messages"][0].content)

In [ ]:
query = "근육량을 늘리려면 어떻게 해야해? 요약된 정보를 바탕으로 대답해"

# 두 정보를 결합
combined_input = f"{query} DOCUMENTS: {tool_output}"

# 도구 실행
answer = summarize_and_answer(combined_input)
print(answer)

이렇게 LangGraph의 ToolNode를 활용하면 다양한 기능을 수행하는 도구를 Agent에 연결하고, 사용자의 자연어 입력에 따라 적절한 도구를 선택하여 실행할 수 있습니다. 특히 RAG 시스템을 도구 형태로 구현하면 문서 검색과 요약, 질문 응답 등의 기능을 유연하게 조합할 수 있습니다.